In [20]:
from numpy import sqrt, log, exp, random
from scipy.stats import norm

# Binomial Tree

Let us start by constructing construcing an *additive* binomial tree.This tree can be used to price European and American Call and Put options.

In [7]:
def Additive(K, T, S, sigma, r, N, American, Call):
    #Coefficients
    dt=T/N
    nu=r-0.5*sigma**2
    dxu=sqrt((sigma**2)*dt+(nu*dt)**2)
    dxd=-dxu
    pu=0.5+0.5*(nu*dt/dxu)
    pd=1-pu
    #Precompute constants
    disc=exp(-r*dt)
    dpu=disc*pu
    dpd=disc*pd
    edxud=exp(dxu-dxd)
    edxd=exp(dxd)
    
    StE=[0]*(N+1)
    StA=[0]*(N+1)
    CE=[0]*(N+1)
    PE=[0]*(N+1)
    CA=[0]*(N+1)
    PA=[0]*(N+1)
    #asset prices at maturity N:
    StA[0]=S*exp(N*dxd)
    StE[0]=S*exp(N*dxd)
    for j in range(1,N+1):
        StE[j]=StE[j-1]*edxud
        StA[j]=StA[j-1]*edxud
    #initilise option values at maturity:
    for j in range(1,N+1):
        CE[j]=max(0,StE[j]-K)
        PE[j]=max(0, K-StE[j])
        CA[j]=max(0,StA[j]-K)
        PA[j]=max(0, K-StA[j])
    #Stepping Back   
    for i in range(N,0,-1):
        for j in range(0,i):
            CE[j]=disc*(pu*CE[j+1]+pd*CE[j])
            PE[j]=disc*(pu*PE[j+1]+pd*PE[j])
            CA[j]=dpd*CA[j]+dpu*CA[j+1]
            PA[j]=dpd*PA[j]+dpu*PA[j+1]
            
            StA[j]=StA[j]/edxd
            PA[j]=max(PA[j],K-StA[j])
            CA[j]=max(CA[j], StA[j]-K)
    if American == True and Call == True:
        return CA[0]
    elif American == True and Call == False:
        return PA[0]
    elif American == False and Call == True:
        return CE[0]
    else:
        return PE[0]
#    return CE[0], PE[0], CA[0],PA[0]

print(Additive(100,32/252,100,0.2,0.0051,500, False, False))
print(Additive(100,1,100,0.2,0.06, 3, True, False))

2.8080545758750968
6.1621091990310015


# Trinomial Tree

Similary, we can implement a trinomial tree to price European, American Call and Put options.

In [6]:
def Trinomial(K, T, S, sigma, r, div, N, American, Call):
#Constants
    dt=T/N
    dx=sigma*sqrt(3*dt)
    nu=r-div-0.5*sigma**2
    edx=exp(dx)
    pu=0.5*((dt*sigma**2+nu**2*dt**2)/(dx**2)+nu*dt/dx)
    pm=1-((dt*sigma**2+nu**2*dt**2)/(dx**2))
    pd=0.5*((dt*sigma**2+nu**2*dt**2)/(dx**2)-nu*dt/dx)
    disc=exp(-r*dt)
#Initialise Asset Prices
    StE=[0]*(2*N+1)
    StA=[0]*(2*N+1)
    CE=[[0 for j in range(2*N+1)] for i in range(N+1)]
    PE=[[0 for j in range(2*N+1)] for i in range(N+1)]
    CA=[[0 for j in range(2*N+1)] for i in range(N+1)]
    PA=[[0 for j in range(2*N+1)] for i in range(N+1)]

    StE[0]=S*exp(-N*dx)
    StA[0]=S*exp(-N*dx)
    for j in range(1,2*N+1):
        StE[j]=StE[j-1]*edx
        StA[j]=StA[j-1]*edx
    for j in range(0,2*N+1):
        CE[N][j]=max(0, StE[j]-K)
        PE[N][j]=max(0,K-StE[j])
        CA[N][j]=max(0, StA[j]-K)
        PA[N][j]=max(0,K-StA[j]-K)
#Stepping back:
    for i in range(N-1,-1,-1):
        for j in range(N-i,N+i+1):
            CE[i][j]=disc*(pu*CE[i+1][j+1]+pm*CE[i+1][j]+pd*CE[i+1][j-1])
            PE[i][j]=disc*(pu*PE[i+1][j+1]+pm*PE[i+1][j]+pd*PE[i+1][j-1])
            CA[i][j]=disc*(pu*CA[i+1][j+1]+pm*CA[i+1][j]+pd*CA[i+1][j-1])
            PA[i][j]=disc*(pu*PA[i+1][j+1]+pm*PA[i+1][j]+pd*PA[i+1][j-1])

            PA[i][j]=max(PA[i][j],K-StA[i])
            CA[i][j]=max(CA[i][j], StA[i]-K)
    if American==True and Call==True:
        return CA[0][N]
    elif American==True and Call==False:
        return PA[0][N]
    elif American==False and Call==True:
        return CE[0][N]
    else:
        return PE[0][N]
    
print(Trinomial(100,1,100,0.2,0.06,0,300,True, True))

10.983406625431213


## Exotic options

We can also use the trees to price exotic options. Below are some examples.

### American Down-and-out option

In [9]:
def ADOBarrier(K,T,S,sigma,r,N, H):
   #Coefficients
   dt=T/N
   nu=r-0.5*sigma**2
   dxu=sqrt((sigma**2)*dt+(nu*dt)**2)
   dxd=-dxu
   pu=0.5+0.5*(nu*dt/dxu)
   pd=1-pu
   #Precompute constants
   disc=exp(-r*dt)
   edxud=exp(dxu-dxd)
   edxd=exp(dxd)

   StA=[0]*(N+1)

   CA=[0]*(N+1)
   #asset prices at maturity N:
   StA[0]=S*exp(N*dxd)
   for j in range(1,N+1):
       StA[j]=StA[j-1]*edxud
   #initilise option values at maturity:

   for j in range(1,N+1):
       if StA[j]> H:
           CA[j]=max(0,StA[j]-K)

       else:
           CA[j]=0

   #Stepping Back   
   for i in range(N,0,-1):
       for j in range(0,i):
           StA[j]=StA[j]/edxd
           if StA[j]> H:
               CA[j]=disc*(pu*CA[j+1]+pd*CA[j])
               CA[j]=max(CA[j],StA[j]-K)
           else:
               CA[j]=0

   return CA[0]

print(ADOBarrier(100,1,100,0.2,0.06,3,95))

9.995775102630393


### Euorpean Up-and-Out Call


In [10]:
def EUOBarrier(K,T,S,sigma,r,N, H):
    #Coefficients
    dt=T/N
    nu=r-0.5*sigma**2
    dxu=sqrt((sigma**2)*dt+(nu*dt)**2)
    dxd=-dxu
    pu=0.5+0.5*(nu*dt/dxu)
    pd=1-pu
    #Precompute constants
    disc=exp(-r*dt)
    edxud=exp(dxu-dxd)
    
    StE=[0]*(N+1)
    CE=[0]*(N+1)

    StE[0]=S*exp(N*dxd)
    for j in range(1,N+1):
        StE[j]=StE[j-1]*edxud
    #initilise option values at maturity:
    
    for j in range(1,N+1):
        if StE[j]< H:
            CE[j]=max(0,StE[j]-K)
        else:
            CE[j]=0
#    for j in range(1,N+1):
#        CE[j]=max(0,StE[j]-K)

    #Stepping Back      
    for i in range(N,0,-1):
        for j in range(0,i):
            StE[j]=StE[j]/exp(-dxu)
            if StE[j]< H:
                CE[j]=disc*(pu*CE[j+1]+pd*CE[j])
            else:
                CE[j]=0
    return CE[0]
print(EUOBarrier(10,0.3,10,0.2,0.01,3000, 11))

0.05420867558544497


An analytical formula exists for pricing some of the exotic options such as the European Up-and-Out Call. (See Niklas Westermark. *Barrier Option Pricing*, Degree Project in Mathematics, First Level. KTH Royal Institute of Technology, Stockholm, Sweden. for more details)
 We can compare the performance of the model with this formula.

To use this formala, first we need to define the BSM formulas

In [14]:
def norm_pdf(x):
    return (1/((2*pi)**0.5))*exp(-0.5*x*x)


def d_j(j, S, K, r, v, T):
    """d_j = \frac{log(\frac{S}{K})+(r+(-1)^{j-1} \frac{1}{2}v^2)T}{v sqrt(T)}"""
    return (log(S/K) + (r + ((-1)**(j-1))*0.5*v*v)*T)/(v*(T**0.5))


def call_price(S, K, r, v, T):
    """Price of a European call option struck at K, with spot S, constant rate r, constant vol v (over the life of the option) and time to maturity T"""
    return S*norm.cdf(d_j(1, S, K, r, v, T))-K*exp(-r*T) * norm.cdf(d_j(2, S, K, r, v, T))


def put_price(S, K, r, v, T):
    """Price of a European put option struck at K, with spot S, constant rate r, constant vol v (over the life of the option) and time to maturity T"""
    return -S*norm.cdf(-d_j(1, S, K, r, v, T))+K*exp(-r*T) * norm.cdf(-d_j(2, S, K, r, v, T))


In [15]:
def AnalUO(S,H,K,r,T,sigma):
    nu=r-0.5*sigma**2
    u=(call_price(S,K,r,sigma,T)-call_price(S,H,r,sigma,T)-(H-K)*exp(-r*T)*norm.cdf(d_j(1, S, H, r, sigma, T))
    -((H/S)**(2*nu/(sigma**2)))*((call_price((H**2/S),K,r,sigma,T))
    -call_price((H**2/S),H,r,sigma,T)-(H-K)*exp(-r*T)*norm.cdf(d_j(1, H, S, r, sigma, T))))
    return u

   

An example:

In [16]:
K=10
T=0.3
S=10
sigma=0.2
r=0.01
H=11
print(AnalUO(S,H,K,r,T,sigma)) 

0.050317307441203614


### European Up-and-In Call

In [17]:
def AnalUI(S,H,K,r,T,sigma):
    nu=r-0.5*sigma**2
    U=(((H/S)**(2*nu/(sigma**2)))*(put_price((H**2/S),K,r,sigma,T)
    -put_price((H**2/S),H,r,sigma,T)+(H-K)*exp(-r*T)*norm.cdf(-d_j(1, H, S, r, sigma, T)))
    +call_price(S,H,r,sigma,T)+(H-K)*exp(-r*T)*norm.cdf(d_j(1, S, H, r, sigma, T)))
    return U
    
print(AnalUI(S,H,K,r,T,sigma))          

0.4009703374396949
0.40097033743969507


The price can also be calculated using the put-call parity

In [18]:
print(call_price(S,K,r,sigma,T)-AnalUO(S,H,K,r,T,sigma))

0.40097033743969507


### American Up-and-In Put

In [19]:
def AUIPBarrier(K,T,S,sigma,r,N, H):
    #Coefficients
    dt=T/N
    nu=r-0.5*sigma**2
    dxu=sqrt((sigma**2)*dt+(nu*dt)**2)
    dxd=-dxu
    pu=0.5+0.5*(nu*dt/dxu)
    pd=1-pu
    #Precompute constants
    disc=exp(-r*dt)
    edxud=exp(dxu-dxd)
    edxd=exp(dxd)

    StA=[0]*(N+1)

    CA=[0]*(N+1)
    #asset prices at maturity N:
    StA[0]=S*exp(N*dxd)
    for j in range(1,N+1):
        StA[j]=StA[j-1]*edxud
    #initilise option values at maturity:
    j2=[i for i in StA if i <H]    
    if len(j2)==len(StA):
        return 0
        #CA[0]=0
        #return CA[0]
    else:
        for j in range(1,N+1):
            CA[j]=max(0,K-StA[j])
#    for j in range(1,N+1):
#        CA[j]=max(0,K-StA[j])
        
    #Stepping Back   
    for i in range(N,0,-1):
        for j in range(0,i):
            CA[j]=disc*(pu*CA[j+1]+pd*CA[j])
            StA[j]=StA[j]/edxd
            CA[j]=max(CA[j],K-StA[j])

    return CA[0]
N=200
print(AUIPBarrier(K,T,S,sigma,r,N, H))

0.4227416365560542
